In [6]:
from datasets import load_dataset, Dataset
import csv
import tqdm.auto as tqdm

In [2]:
data = load_dataset("mandarjoshi/trivia_qa", 'rc')

Using the latest cached version of the dataset since mandarjoshi/trivia_qa couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'rc' at /private/home/ziweiji/.cache/huggingface/datasets/mandarjoshi___trivia_qa/rc/0.0.0/0f7faf33a3908546c6fd5b73a660e0f8ff173c2f (last modified on Thu Jan  9 16:41:23 2025).


Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [7]:
# There are some repated question_id ids in the dataset, so we need to remove them

train_data = data['train']
train_df = train_data.to_pandas()
train_df = train_df.drop_duplicates(subset='question_id')
train_data = Dataset.from_pandas(train_df)
train_data = train_data.shuffle(seed=42).select(range(10000))

validation_data = data['validation']
validation_df = validation_data.to_pandas()
validation_df = validation_df.drop_duplicates(subset='question_id')
validation_data = Dataset.from_pandas(validation_df)
test_val_data = validation_data.shuffle(seed=42).select(range(2000))
test_data = test_val_data.select(range(1000))
val_data = test_val_data.select(range(1000, 2000))

In [8]:
def write_file(data, split):
    all_ids = set()
    with open(f"sampled/{split}.csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "question", "answer"])
        for i in range(len(data)):
            id = data[i]['question_id']
            assert id not in all_ids
            all_ids.add(id)
            writer.writerow([id, data[i]['question'], data[i]['answer']['aliases']])

In [9]:
write_file(train_data, "train")
write_file(val_data, "val")
write_file(test_data, "test")